In [1]:
# files/modules
from env import host, user, password
import acquire as a

# manipulations
import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

## Acquire
'Basic' columns and then all of the columns for Zillow data (that meets parameters)

In [2]:
zillow = a.basic_acquire_zillow()

Acquiring data from SQL db


In [3]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   bedrooms   52441 non-null  float64
 1   bathrooms  52441 non-null  float64
 2   area       52359 non-null  float64
 3   tax_value  52440 non-null  float64
dtypes: float64(4)
memory usage: 1.6 MB


In [4]:
zillow_l = a.larger_acquire_zillow()

Acquiring data from SQL db


In [5]:
zillow_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   propertylandusetypeid         52441 non-null  float64
 2   id                            52441 non-null  int64  
 3   airconditioningtypeid         13638 non-null  float64
 4   architecturalstyletypeid      70 non-null     float64
 5   basementsqft                  47 non-null     float64
 6   bathroomcnt                   52441 non-null  float64
 7   bedroomcnt                    52441 non-null  float64
 8   buildingclasstypeid           0 non-null      object 
 9   buildingqualitytypeid         33740 non-null  float64
 10  calculatedbathnbr             52304 non-null  float64
 11  decktypeid                    389 non-null    float64
 12  finishedfloor1squarefeet      4381 non-null   float64
 13  c

## Preparation/Splitting


In [8]:
# looking at null counts
zillow.isnull().sum()

bedrooms      0
bathrooms     0
area         82
tax_value     1
dtype: int64

In [9]:
zillow_l.isnull().sum()

parcelid                        0
propertylandusetypeid           0
id                              0
airconditioningtypeid       38803
architecturalstyletypeid    52371
                            ...  
censustractandblock           123
propertylandusedesc             0
id                              0
logerror                        0
transactiondate                 0
Length: 63, dtype: int64

In [10]:
# The nulls for aircondition and architecture are so large, going to drop columns entirely. Also dropping 
# landtaxvaluedollarcnt, structuretaxvaluedollarcnt, and taxamount for target variable (taxvaluedollarcnt) integrity.

In [11]:
zillow_l = zillow_l.drop(columns= ['airconditioningtypeid', 'architecturalstyletypeid', 'landtaxvaluedollarcnt',
                                  'structuretaxvaluedollarcnt', 'taxamount'])

In [13]:
zillow_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   propertylandusetypeid         52441 non-null  float64
 2   id                            52441 non-null  int64  
 3   basementsqft                  47 non-null     float64
 4   bathroomcnt                   52441 non-null  float64
 5   bedroomcnt                    52441 non-null  float64
 6   buildingclasstypeid           0 non-null      object 
 7   buildingqualitytypeid         33740 non-null  float64
 8   calculatedbathnbr             52304 non-null  float64
 9   decktypeid                    389 non-null    float64
 10  finishedfloor1squarefeet      4381 non-null   float64
 11  calculatedfinishedsquarefeet  52359 non-null  float64
 12  finishedsquarefeet12          52194 non-null  float64
 13  f